In [37]:
import numpy 
import pandas as pd
import dice_ml
import xgboost as xgb
import warnings
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

warnings.filterwarnings("ignore") 
pd.options.display.max_rows = 500

In [38]:
TARGET = 'TSV'

In [39]:
data = pd.read_csv('../data/TotalClothingValue.csv', index_col=0)
data = data.dropna()

In [40]:
mapping = {-2: 0, -1: 1, 0: 2, 1: 3, 2: 4}
data[TARGET] = data[TARGET].replace(mapping)

In [51]:
columns = ['TSV', 'TPV', 'TCV', 'TSL']
columns.remove(TARGET)

In [54]:
# Working only on TSV now
data = data.drop(columns=columns)

In [55]:
features = data.drop(TARGET, axis=1).columns.tolist()
target = data[TARGET]

In [56]:
datasetX = data.drop(TARGET, axis=1)

In [57]:
x_train, x_test, y_train, y_test = train_test_split(datasetX,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=0)

In [58]:
categorical_features = x_train.columns.difference(features)

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, features),
        ('cat', categorical_transformer, categorical_features)])

regr = Pipeline(steps=[('preprocessor', transformations),
                        ('classifier', xgb.XGBClassifier())])
model = regr.fit(x_train, y_train)

In [59]:
y_pred = model.predict(x_test)

In [60]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.6519607843137255


In [8]:
import pickle
#pickle.dump(model, open('../models/tsv+3_full.pkl', 'wb'))
#model = pickle.load(open('../models/tsv+3_full.pkl', 'rb'))

In [61]:
d = dice_ml.Data(dataframe=data, continuous_features=features, outcome_name=TARGET)
m = dice_ml.Model(model=model, backend='sklearn', model_type='classifier')

In [62]:
exp = dice_ml.Dice(d, m, method='random')

In [76]:
always_immutable = ['DAY', 'AvgMaxDailyTemp','AvgMinDailyTemp','School','StartTime', 'SchoolType']
freezed = always_immutable + []

features_to_vary = data.columns.difference(freezed).to_list()

In [77]:
features_to_vary.remove(TARGET)

In [65]:
query_instances = x_test[:]


In [ ]:
query_instances

In [78]:
cf = exp.generate_counterfactuals(query_instances=query_instances, total_CFs=4, desired_class=3, features_to_vary=features_to_vary)

  0%|          | 0/408 [00:00<?, ?it/s]

  2%|▏         | 10/408 [00:06<04:10,  1.59it/s]

No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec


  3%|▎         | 11/408 [00:06<04:47,  1.38it/s]

Only 1 (required 4)  Diverse Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec


...

 94%|█████████▍| 384/408 [03:40<00:14,  1.68it/s]

Only 2 (required 4)  Diverse Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec


 99%|█████████▊| 402/408 [03:52<00:04,  1.38it/s]

No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec


100%|██████████| 408/408 [03:56<00:00,  1.72it/s]


In [ ]:
cf.visualize_as_dataframe(show_only_changes=True)

In [79]:
r = []
for i in range(len(x_test)):
    r.append(x_test[i:i+1])
    if cf.cf_examples_list[i].final_cfs_df is not None:
        r.append(cf.cf_examples_list[i].final_cfs_df)

r2 = pd.concat(r)

In [80]:
mapping = {0: -2, 1: -1, 2: 0, 3: 1, 4: 2}
r2[TARGET] = r2[TARGET].replace(mapping)

In [81]:
r2

,DAY,School,SchoolType,StartTime,AvgMaxDailyTemp,AvgMinDailyTemp,AvgIndoorRelativeHumidity,IndoorTempDuringSurvey,Grade,Age,Gender,FormalClothing,TotalCLOwithChair,SwC,MC,TSV
939,1.0,3.0,1.0,3.0,22.4,10.1,46.3,17.0,5.0,10.0,0.0,1.0,1.16,1.0,4.0,NaN
0,1.0,3.0,1.0,3.0,22.4,10.1,71.6,13.3,5.0,10.0,0.0,0.0,1.58,1.0,4.0,1.0
1,1.0,3.0,1.0,3.0,22.4,10.1,74.7,12.8,3.0,8.0,0.0,1.0,1.56,1.0,4.0,1.0
2,1.0,3.0,1.0,3.0,22.4,10.1,46.3,14.0,3.0,10.0,1.0,1.0,1.58,1.0,3.0,1.0
3,1.0,3.0,1.0,3.0,22.4,10.1,69.5,14.0,5.0,10.0,0.0,1.0,1.56,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,2.0,4.0,1.0,1.0,19.6,6.6,59.6,13.1,3.0,7.0,1.0,1.0,1.33,2.0,4.0,NaN
0,2.0,4.0,1.0,1.0,19.6,6.6,59.6,13.1,3.0,7.0,0.0,0.0,1.58,1.0,4.0,1.0
1,2.0,4.0,1.0,1.0,19.6,6.6,53.4,13.1,4.0,7.0,1.0,0.0,1.56,1.0,4.0,1.0
2,2.0,4.0,1.0,1.0,19.6,6.6,45.5,13.1,5.0,7.0,1.0,1.0,1.56,1.0,4.0,1.0


In [87]:
r2.to_csv(f'../results/{TARGET}/Total_all.csv')